Let's do a simple neuron only with Python:

In [70]:
import math

inputs = [.5, .3, .2]
ws= [.4, .7, .2]

#basically this just calculates the Sigmoid fuction
def sigmoid(x):
    y= 1.0 / (1 + math.exp(-x))
    return y

#This is the activation of the neuron
def activate(inputs,ws):
    sum=0
    for input,w in zip(inputs,ws):
        sum += input*w
    return sigmoid(sum)

output= activate(inputs, ws)
print(output)

0.610639233949222


Let's use the dot notation using numpy and also using the sigmoid function directly from tensorflow:

In [71]:
import numpy as np
import tensorflow as tf

inputs = [.5, .3, .2]
ws= [.4, .7, .2]

#this does the dot notation using numpy
# and multiplies the vectors directly
sum=np.dot(inputs,ws)

#we can just directly use the sigmoid function
#from tensorflow
output=tf.sigmoid(sum)

print(output)

tf.Tensor(0.6106392339492219, shape=(), dtype=float64)


Now let's build a neural network (a multi layer perception model) only with Python and NumPy:

In [72]:
class MLP:
    def __init__(self, num_inputs = 3, num_hidden=[3, 5], num_outputs=2):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs

        layers = [num_inputs] + num_hidden + [num_outputs]

        self.weights = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            self.weights.append(w)


    def forward_propogate(self,inputs):
        activations = inputs

        for w in self.weights:
            net_inputs = np.dot(activations, w)

            activations = self.sigmoid(net_inputs)
        return activations


    def sigmoid(self, x):
        y = 1.0 / (1 + np.exp(-x))
        return y

So no that we made the NN object, lets create one MLP, give it some inputs, run the prop function and print the result:

In [73]:
mlp = MLP()
inputs = np.random.rand(mlp.num_inputs)
outputs = mlp.forward_propogate(inputs)

print("The network input is: {}".format(inputs))
print("The network output is: {}".format(outputs))

The network input is: [0.75706851 0.82002174 0.44752078]
The network output is: [0.8967814  0.75884141]
